In [1]:
import os
import json

In [2]:
def read_py_file(file_path):
    """Odczytuje zawartość pliku .py jako tekst."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

In [3]:
def read_ipynb_file(file_path):
    """Odczytuje i łączy komórki kodu z pliku .ipynb."""
    with open(file_path, 'r', encoding='utf-8') as f:
        notebook = json.load(f)
    
    code_content = []
    for cell in notebook.get('cells', []):
        if cell.get('cell_type') == 'code':
            code_content.append(''.join(cell.get('source', [])))
    return "\n# --- Koniec komórki ---\n".join(code_content)

In [4]:
def consolidate_project_to_txt(root_dir='.', output_file='caly_projekt.txt'):
    """
    Zbiera zawartość plików .py i .ipynb w projekcie do jednego pliku tekstowego.
    """
    # Zestawy nazw do zignorowania - osobno dla katalogów i plików
    ignore_dirs = {'.git', '.venv', 'venv', '__pycache__', '.ipynb_checkpoints'}
    ignore_files = {'Agents_beta (10).py','pack_project.ipynb', 'caly_projekt.txt'}

    with open(output_file, 'w', encoding='utf-8') as outfile:
        # Używamy os.walk z topdown=True (domyślnie), co pozwala modyfikować dirnames
        for dirpath, dirnames, filenames in os.walk(root_dir):
            
            # === KLUCZOWA POPRAWKA ===
            # Modyfikujemy listę `dirnames` W MIEJSCU, aby os.walk nie wchodził
            # do katalogów, które chcemy zignorować.
            dirnames[:] = [d for d in dirnames if d not in ignore_dirs]
            
            for filename in sorted(filenames):
                if filename in ignore_files:
                    continue
                
                if not (filename.endswith('.py') or filename.endswith('.ipynb')):
                    continue

                relative_path = os.path.join(dirpath, filename)
                normalized_path = os.path.normpath(relative_path).replace(os.sep, '/')

                outfile.write(f"--- FILE: {normalized_path} ---\n\n")
                
                try:
                    content = ""
                    if filename.endswith('.py'):
                        content = read_py_file(relative_path)
                    elif filename.endswith('.ipynb'):
                        content = read_ipynb_file(relative_path)
                    
                    outfile.write(content)
                    outfile.write("\n\n\n")
                except Exception as e:
                    outfile.write(f"### BŁĄD ODCZYTU PLIKU: {e} ###\n\n\n")
                            
    print(f"✅ Plik '{output_file}' został pomyślnie wygenerowany.")

In [5]:
if __name__ == '__main__':
    consolidate_project_to_txt()

✅ Plik 'caly_projekt.txt' został pomyślnie wygenerowany.
